In [95]:
import emails
from emails.template import JinjaTemplate as T
from PIL import Image
from lxml import html
from itsdangerous import URLSafeTimedSerializer
from loguru import logger

In [98]:
MOKOMO_MAIL_ADDRESS = "mokomo.pro@gmail.com"
MOKOMO_MAIL_PWD = "6vkpTFgV@v#DU&"

CONF_TOKEN_SECRET_KEY = "0ce1091d-3d54-4bc6-b51a-91b804168dad"
CONF_TOKEN_PASSWORD_SALT = "baa66e1b-2fd4-4c5f-8150-1004843060eb"

In [100]:
def generate_confirmation_token(id_to_encode: str):
    serializer = URLSafeTimedSerializer(CONF_TOKEN_SECRET_KEY)
    return serializer.dumps(id_to_encode, salt=CONF_TOKEN_PASSWORD_SALT)


def confirm_token(token):
    serializer = URLSafeTimedSerializer(CONF_TOKEN_SECRET_KEY)
    try:
        decoded_id = serializer.loads(
            token,
            salt=CONF_TOKEN_PASSWORD_SALT
        )
    except:
        logger.info("token is not valid")
        return False
    return decoded_id

In [101]:
def add_unsubscribe_link(html_template: str, contact_id: str, target_base_url: str) -> str:
    token = generate_confirmation_token(contact_id)
    unsubscribe_url = target_base_url + f"/{token}"
    unsubscribe_link = f"""
    <div class="footer" style="clear: both; margin-top: 30px; width: 100%; font-size: 10px;">
        <br> Vous ne souhaitez plus recevoir ces emails ? <a href={unsubscribe_url} style="text-decoration: underline; color: #999999; text-align: center;">Se désinscrire</a>.
    </div>"""
    element = html.fromstring(unsubscribe_link)
    html_template_tree = html.fromstring(html_template)
    html_template_tree.body.append(element)
    return html.tostring(html_template_tree).decode("utf-8")

In [102]:
def add_pixel_link(html_template: str, email_id: str, target_base_url: str):
    token = generate_confirmation_token(email_id)
    image_url = target_base_url + f"/{token}"
    pixel_link = f"<img src={image_url} height='0px' width='0px'/>"
    element = html.fromstring(pixel_link)
    html_template_tree = html.fromstring(html_template)
    html_template_tree.body.append(element)
    return html.tostring(html_template_tree).decode("utf-8")

In [103]:
def send_test_email(user_email: str, html_template: str):
    message = emails.html(subject=T('Votre compte Mokomo a été créé'),
                          html=T(html_template),
                          mail_from=('Mokomo', MOKOMO_MAIL_ADDRESS))
        
    message.send(
        to=user_email,
        smtp={
            "host": "smtp.gmail.com",
            "port": 465,
            "ssl": True,
            "user": MOKOMO_MAIL_ADDRESS,
            "password": MOKOMO_MAIL_PWD})

In [104]:
html_template = """
<html>
    <body>
        <div style="font-size: 14px;">
            <p>
                Bonjour,<br><br>
                Merci d’avoir rejoint Mokomo.<br><br>
                L’équipe Mokomo.
            </p>
        </div>
    </body>
</html>
"""

In [112]:
html_templated_unsubscribe = add_unsubscribe_link(html_template, contact_id="lmanhes", target_base_url="http://127.0.0.1:8000/unsubscribe")

In [113]:
html_templated_pixel = add_pixel_link(html_templated_unsubscribe, email_id="YOOOOO", target_base_url="http://127.0.0.1:8000/pixel")

In [114]:
html_templated_pixel

'<html>\n    <body>\n        <div style="font-size: 14px;">\n            <p>\n                Bonjour,<br><br>\n                Merci d&#8217;avoir rejoint Mokomo.<br><br>\n                L&#8217;&#233;quipe Mokomo.\n            </p>\n        </div>\n    <div class="footer" style="clear: both; margin-top: 30px; width: 100%; font-size: 10px;">\n        <br> Vous ne souhaitez plus recevoir ces emails ? <a href="http://127.0.0.1:8000/unsubscribe/ImxtYW5oZXMi.YpXIkQ.DHKjyaHlJyt8PDaYAH1ag6vFQBU" style="text-decoration: underline; color: #999999; text-align: center;">Se d&#233;sinscrire</a>.\n    </div><img src="http://127.0.0.1:8000/pixel/IllPT09PTyI.YpXIkQ.9yTEhSISzi-X-8DDaO_PKjACzxs" height="0px" width="0px"></body>\n</html>'

In [115]:
send_test_email(user_email="louismanhes@icloud.com", html_template=html_templated_pixel)